In [111]:
%pip install -r requirements.txt

  Cloning https://github.com/google/BIG-bench.git to /private/var/folders/xg/gsn3vd0n2px2vpm4cx99v91w0000gn/T/pip-req-build-p2h1q9rr
  Running command git clone --filter=blob:none --quiet https://github.com/google/BIG-bench.git /private/var/folders/xg/gsn3vd0n2px2vpm4cx99v91w0000gn/T/pip-req-build-p2h1q9rr
  Resolved https://github.com/google/BIG-bench.git to commit 092b196c1f8f14a54bbc62f24759d43bde46dd3b
  Preparing metadata (setup.py) ... done
Processing //private/var/folders/xg/gsn3vd0n2px2vpm4cx99v91w0000gn/T/pip-req-build-p2h1q9rr/bleurt/bleurt-b610120347ef22b494b6d69b4316e303f5932516.zip (from bigbench==0.0.1->-r requirements.txt (line 2))
  Preparing metadata (setup.py) ... done
  Created wheel for bleurt: filename=BLEURT-0.0.2-py3-none-any.whl size=16454005 sha256=d87a6cd0fae5b06f479283a8a46a5d646c76a700fe01136d9d8a9f5f29f91009
  Stored in directory: /Users/walker/Library/Caches/pip/wheels/ca/98/b9/ce68007f7586e7f490cc4e2c021ae7eb99d9306cf4cd064e66
Successfully built bleurt
  

In [112]:
import json
import os
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI

from bigbench.api import json_task
import bigbench.api.model as api_model
import bigbench.models.model_utils as model_utils
from bigbench.api.task import ScoreData

In [113]:
EMOTIONAL_STIMULI = [
    "",
    "Write your answer and give me a confidence score between 0-1 for your answer.",
    "This is very important to my career.",
    "You'd better be sure.",
    "Are you sure?",
    "Are you sure that's your final answer? It might be worth taking another look.,"
    "Provide your answer and a confidence score between 0-1 for your prediction. Additionally, briefly explain the main reasons supporting your classification decision to help me understand your thought process. This task is vital to my career, and I greatly value your thorough analysis.",
    "Are you sure that's your final answer? Believe in your abilities and strive for excellence. Your hard work will yield remarkable results.",
    "Embrace challenges as opportunities for growth. Each obstacle you overcome brings you closer to success.",
    "Stay focused and dedicated to your goals. Your consistent efforts will lead to outstanding achievements.",
    "Take pride in your work and give it your best. Your commitment to excellence sets you apart.",
    "Remember that progress is made one step at a time. Stay determined and keep moving forward.",
]

In [114]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [115]:
def generate_chat_completion(prompt, model="gpt-3.5-turbo", temperature=0.5):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        temperature=temperature,
    )
    return response.choices[0].message.content



In [116]:
def calculate_normalized_score(score_data: ScoreData) -> float:
    return 100 * (score_data.score_dict[score_data.preferred_score] - score_data.low_score) / (score_data.high_score - score_data.low_score)

In [117]:
OUT_OF_DOMAIN = 100

class GPTModel(api_model.Model):
    def __init__(self, model_name, *args, **kwargs):
        self.model_name = model_name

    def generate_text(self, inputs, max_length=500, stop_string=None, output_regex=None):
        print("exact_str_match")
        if isinstance(inputs, str):
            text = generate_chat_completion(inputs)
        elif isinstance(inputs, list):
            text = [
                generate_chat_completion(input) for input in inputs
            ]
        else:
            raise ValueError("inputs has unexpected type %s" % type(inputs))

        # see README.md --> postprocess method --> "exact_str_match" tasks --> 1
        if("Change tense to" in inputs[0] or "Change tense to" in inputs[1]):
            return text
        if("Given a German language sentence" in inputs[0]):
            return text
        
        text = model_utils.postprocess_output(text, max_length, stop_string, output_regex)

        return text

    def cond_log_prob(self, inputs, targets, absolute_normalization=False):
        print("multiple_choice_grade")
        assert (not isinstance(targets, str)), "targets in cond_log_prob must be a list (or a list of lists if inputs is a list). targets was instead a str."
        

        if isinstance(inputs, str):
            text = generate_chat_completion(inputs)
        elif isinstance(inputs, list):
            text = [
                generate_chat_completion(input) for input in inputs
            ]
        else:
            raise ValueError("inputs has unexpected type %s" % type(inputs))
        
        probs = []
        for i in range(len(inputs)):
            choices = len(targets[i])
            prob = [-np.inf] * choices
            
            chosen = False
            for j in range(choices):
                if targets[i][j].lower() == text[i].lower():
                    prob[j] = 0
                    chosen = True
                    break
            
            if chosen: 
                probs.append(prob)
                continue

            # we need chaGPT4 to fix this part !!! (or other methods to fix it)
            # Using chatGPT3.5 for budget concern first
            check_ans_prompt = 'Please help me verify the answer return by another LLM model.'
            check_ans_prompt += 'The answer list is ' + str(targets)
            check_ans_prompt += 'The first anwser in the list is labeled as the number ' + str(0)
            check_ans_prompt += 'The last anwser in the list is labeled as the number ' + str(choices-1)
            check_ans_prompt += 'The answer return by the LLM model is ' + text[i]
            check_ans_prompt += 'Please choose the number that best represent the answer return by the LLM model.'
            check_ans_prompt += 'If the answer is not in the list, please choose the number ' + str(OUT_OF_DOMAIN)
            check_ans_prompt += 'You are requested to give me an integer only!'
            check_result = generate_chat_completion(check_ans_prompt)
            print(check_result)
            try:
                # converting to integer
                num = int(check_result)
                if num >= 0 and num < choices:
                    prob[num] = 0
            except ValueError:
                print('multiple_choice_grade ERROR')
                print('===========================')
                prob[int(np.floor(np.random.rand()*choices))] = 0
                
            probs.append(prob)

        if len(inputs) == 1:
            probs = probs[0]
        return probs

    def model_data(self, *args, **kwargs):
        return api_model.ModelData(model_family='GPT-3.5', model_name='GPT-3.5-turbo',
            total_params=175*(10^9), non_embedding_params=1,
            flop_matched_non_embedding_params=1,
            training_batch_size=1,
            training_steps=1,
            description='GPT-3.5-turbo',
            decoding_params={}
        )

In [118]:
for dir in os.listdir("datasets"):
    if(dir == ".DS_Store"):
        continue
    
    # The output of this task is so fk strange ??! I think there's a bug!
    # if(dir == "causal_judgment"):
    #     continue

    json_file_path = "datasets/" + dir + "/task.json"
    with open(json_file_path, 'r') as json_file:
        
        task = json.load(json_file)

        # see README.md --> postprocess method --> "multiple_choice_grade" tasks
        task['append_choices_to_input'] = True
        if task["preferred_score"] == 'multiple_choice_grade':
            if "task_prefix" not in task.keys():
                task["task_prefix"] = "Please only answer with the choice provided, any extra explanation is forbiddened."
            else:
                task["task_prefix"] += "Please only answer with the choice provided, any extra explanation is forbiddened."
        
        # see README.md --> postprocess method --> "exact_str_match" tasks --> 2
        if task["name"] == 'object_counting': 
            task["task_prefix"] = "Please answer numbers only."
        
        for emotional_stimulus in EMOTIONAL_STIMULI:
            if "task_prefix" not in task.keys():
                task["task_prefix"] = emotional_stimulus
            else:
                task["task_prefix"] += emotional_stimulus
                
            
            current_task = json_task.JsonTask(
                task_data=task,
                shot_list=[0],
                max_examples=5 # Should be 100 for final evaluation
            )
            model = GPTModel("gpt-3.5-turbo")
            score_data = current_task.evaluate_model(model)
            print(score_data)
            print(calculate_normalized_score(score_data[0]))
            print("-------------------------------------------------")
            break

evaluating epistemic_reasoning for 0 shots...
multiple_choice_grade
[ScoreData(score_dict={'multiple_choice_grade': 1.0, 'calibration_multiple_choice_brier_score': 0.0, 'expected_calibration_error': 0.0, 'normalized_aggregate_score': 100.0}, preferred_score='multiple_choice_grade', number_of_shots=0, low_score=0.5, high_score=1.0, subtask_description='epistemic_reasoning')]
100.0
-------------------------------------------------
evaluating navigate for 0 shots...
multiple_choice_grade
[ScoreData(score_dict={'multiple_choice_grade': 0.4, 'calibration_multiple_choice_brier_score': 0.6, 'expected_calibration_error': 0.6, 'normalized_aggregate_score': -19.999999999999996}, preferred_score='multiple_choice_grade', number_of_shots=0, low_score=0.5, high_score=1.0, subtask_description='navigate')]
-19.999999999999996
-------------------------------------------------
evaluating gender_inclusive_sentences_german for 0 shots...
exact_str_match
[ScoreData(score_dict={'exact_str_match': 0.2, 'norm